In [1]:
import sys
sys.path.append("../src")

In [2]:
from wav2vec2 import Wav2Vec2Config, Wav2Vec2ForCTC, Wav2Vec2Processer
from transformers import Wav2Vec2ForCTC as HFWav2Vec2ForCTC

import tensorflow as tf
import torch
import numpy as np

def get_difference(tf_out, hf_out):
    return np.max(tf_out.numpy() - hf_out.numpy())

In [3]:
batch, _ = tf.audio.decode_wav(tf.io.read_file("../data/sample.wav"))
processor = Wav2Vec2Processer(is_tokenizer=False)
batch = tf.transpose(batch, perm=(1, 0))

batch = tf.concat([batch, batch], axis=0)

batch = processor(batch)
hf_batch = torch.from_numpy(batch.numpy()).float()

batch, hf_batch

(<tf.Tensor: shape=(2, 46797), dtype=float32, numpy=
 array([[ 0.00455413, -0.00263517,  0.00814878, ..., -0.00263517,
         -0.01701376, -0.02779771],
        [ 0.00455413, -0.00263517,  0.00814878, ..., -0.00263517,
         -0.01701376, -0.02779771]], dtype=float32)>,
 tensor([[ 0.0046, -0.0026,  0.0081,  ..., -0.0026, -0.0170, -0.0278],
         [ 0.0046, -0.0026,  0.0081,  ..., -0.0026, -0.0170, -0.0278]]))

In [4]:
tf_model = Wav2Vec2ForCTC.from_pretrained("/Users/vasudevgupta/Local/wav2vec2/wav2vec2-base-960h", input_shape=batch.shape)

Loading weights locally from `/Users/vasudevgupta/Local/wav2vec2/wav2vec2-base-960h`
Total number of loaded variables: 212


In [5]:
hf_model = HFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tf_out = tf_model(batch, training=False)["logits"]

In [7]:
with torch.no_grad():
    hf_out = hf_model(hf_batch)["logits"]

In [8]:
tf_out.shape, hf_out.shape

(TensorShape([2, 145, 32]), torch.Size([2, 145, 32]))

In [9]:
print("difference in logits:", get_difference(tf_out, hf_out))

difference in logits: 0.003186226


In [10]:
def tf_forward(*args, **kwargs):
    return tf_model(*args, **kwargs)
tf_forward = tf.function(tf_forward, autograph=True, jit_compile=True)

In [16]:
tf_out = tf_forward(batch, training=False)["logits"]

In [17]:
print("difference in graph based model logits:", get_difference(tf_out, hf_out))

difference in graph based model logits: 0.0023155212


In [5]:
tf.saved_model.save(tf_model, "dummy")


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: dummy/assets
INFO:tensorflow:Assets written to: dummy/assets


In [7]:
model = tf.saved_model.load("dummy")

In [9]:
@tf.function(autograph=True, jit_compile=True)
def tf_foward(*args, **kwargs):
    return model(*args, **kwargs)

In [10]:
tf_foward(batch)

{'logits': <tf.Tensor: shape=(2, 145, 32), dtype=float32, numpy=
 array([[[ 13.001484 , -26.905651 , -26.621664 , ...,  -6.4049616,
           -8.698974 ,  -7.6948757],
         [ 13.019491 , -26.890331 , -26.60231  , ...,  -6.399632 ,
           -8.719545 ,  -7.7269363],
         [ 13.018565 , -26.89125  , -26.603218 , ...,  -6.4000196,
           -8.720039 ,  -7.727183 ],
         ...,
         [ 13.018611 , -26.891388 , -26.603424 , ...,  -6.399854 ,
           -8.719286 ,  -7.728012 ],
         [ 13.016644 , -26.893538 , -26.605778 , ...,  -6.3998675,
           -8.715607 ,  -7.730587 ],
         [ 13.01872  , -26.890413 , -26.602463 , ...,  -6.399314 ,
           -8.718278 ,  -7.7274504]],
 
        [[ 13.001543 , -26.905685 , -26.6217   , ...,  -6.4050837,
           -8.699371 ,  -7.6951613],
         [ 13.0193405, -26.8908   , -26.60276  , ...,  -6.3998156,
           -8.72005  ,  -7.7267733],
         [ 13.018505 , -26.890938 , -26.602905 , ...,  -6.3999214,
           -8.72013